In [3]:
%pip install tabula
import pandas as pd
import tabula
import numpy as np


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
new_horaires_début_df=pd.read_csv(r"/workspaces/metro-JO/plages_hor_debut_event.csv", delimiter=',')
new_horaires_fin_df= pd.read_csv(r"/workspaces/metro-JO/plages_hor_fin_event.csv", delimiter=',')

In [5]:
# Ajout du script Nb de personnes + Arrêts + Lignes des lieux des événements

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# %% [markdown]
# Liste des lieux et url 

# %%

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_lines = {
            'Capacité': [],
            'Stations': []  # Création d'une liste pour stocker les stations
        }
        for paragraph in soup.find_all('p'):
            # Recherche des lignes contenant les Capacité spécifiques
            if ('Capacité' in paragraph.text or 
                'Capacité court Philippe-Chatrier' in paragraph.text or 
                'Volleyball' in paragraph.text or 
                'Pour la gymnastique rythmique' in paragraph.text):
                specific_lines['Capacité'].append(paragraph.text.strip())
                
            # Extraction des phrases après "desservie" ou "desservi"
            if 'desservie' in paragraph.text or 'desservi' in paragraph.text:
                sentences = paragraph.text.split('.')  # Séparation en phrases
                for sentence in sentences:
                    if 'desservie' in sentence or 'desservi' in sentence:
                        specific_lines['Stations'].append(sentence.strip())
        
        return specific_lines
    return None


# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Infos'])
# Expansion du dictionnaire pour créer les colonnes 'Capacité' et 'Stations'
df = pd.concat([df.drop(['Infos'], axis=1), df['Infos'].apply(pd.Series)], axis=1)


# Compléter pour les données manquantes : Concorde 

# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[6, 1]=texte_places.strip()

# Compléter pour les données manquantes : Arena Paris Sud

#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[8, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

# %%
def extraire_nombres(texte):
    nombres = re.findall(r'[-+]?\d*\.\d+|\d+', str(texte))  # Expression régulière pour les nombres entiers ou décimaux
    return ' '.join(nombres)  # Retourne les nombres extraits sous forme d'une chaîne séparée par des espaces

# Appliquer la fonction sur la colonne 'Capacité'
df['Nombre de personnes à l\'événément'] = df['Capacité'].apply(extraire_nombres)

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer "202" par une chaîne vide dans la colonne 'Nombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace('202', '')

# Supprimer les espaces dans la colonne 'NNombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace(' ', '')

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer les valeurs dans la colonne 'Nombres' aux lignes spécifiques
df.loc[2, 'Nombre de personnes à l\'événément'] = '8000'  # Remplacer la deuxième ligne par '8000'
df.loc[10, 'Nombre de personnes à l\'événément'] = '6700'  # Remplacer la 10ème ligne par '6700'
df.loc[14, 'Nombre de personnes à l\'événément'] = '6000'  # Remplacer la 14ème ligne par '6000'

# Fonction pour supprimer le zéro en tête des nombres
def remove_leading_zero(number):
    if number.startswith('0'):
        return number[1:]
    return number

# Appliquer la fonction à la colonne 'Nombres' pour supprimer les zéros en tête
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].astype(str).apply(remove_leading_zero)


arrêts=[['BIR HAKEIM', 'ALMA MARCEAU'],
        ['LA MOTTE-PICQUET-GRENELLE', 'LA MOTTE-PICQUET-GRENELLE', 'SEGUR'], 
        ['FRANKLIN D. ROOSEVELT', 'FRANKLIN D. ROOSEVELT'], 
        ['INVALIDES'], 
        ['INVALIDES'],
        ['IENA','TROCADERO'], 
        ['PALAIS-ROYAL','PALAIS-ROYAL','MADELEINE', 'MADELEINE', 'MADELEINE', 'OPERA'],
        ['PORTE DE SAINT-CLOUD','PORTE D\'AUTEUIL'], 
        ['PORTE DE VERSAILLES','BALARD','PORTE DE VANVES'], 
        ['BERCY','BERCY','GARE DE LYON'],
        ['PORTE DE LA CHAPELLE'],
        ['LA DEFENSE'],
        ['x'],
        ['SAINT-DENIS-PORTE DE PARIS'], 
        ['x'],
        ['x']]
 
df['Arrêts']=arrêts

# Fonction pour extraire les nombres après le mot "Métro"
def extract_numbers(text):
    matches = re.findall(r'Métro\s+(\d+)', str(text), flags=re.IGNORECASE)
    return matches

# Appliquer la fonction à la colonne 'Stations' pour créer une nouvelle colonne 'Nombres après Métro'
df['Lignes'] = df['Stations'].apply(extract_numbers)

# Compléter pour Franklin D
df.at[2, 'Lignes'] = [6, 9]

# Supprimer les colonnes 'Capacité' et 'Stations'
colonnes_a_supprimer = ['Capacité', 'Stations']
df = df.drop(colonnes_a_supprimer, axis=1)

# Calculer le nombre d'éléments dans chaque vecteur de la ligne 'Lignes'
df['Nombre de lignes'] = df['Lignes'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Convertir la colonne 'Nombre de personnes à l\'événément' en type numérique (int ou float)
df['Nombre de personnes à l\'événément'] = pd.to_numeric(df['Nombre de personnes à l\'événément'], errors='coerce')

# Créer la nouvelle colonne en calculant la division
df['Nombre de personnes par ligne'] = df['Nombre de personnes à l\'événément'] / df['Nombre de lignes']

# Définir l'option d'affichage pour éviter la notation scientifique
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# Identifier les indices des lignes contenant les valeurs spécifiques dans la colonne 'lieu'
indices_a_supprimer = df[df['Lieu'].isin(['centre-aquatique', 'site-escalade-bourget', 'arena-paris-nord'])].index

# Supprimer les lignes correspondantes
df = df.drop(indices_a_supprimer)

display(df)

df.to_excel('df.xlsx', index=False)

# Copie du programme fréquentation_metro

# Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# %%
# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)


# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)

/tmp/ipykernel_2430/3461944228.py:89: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texte_places = soup.find(text=lambda text: text and "places" in text)
/tmp/ipykernel_2430/3461944228.py:108: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)


,Lieu,Nombre de personnes à l'événément,Arrêts,Lignes,Nombre de lignes,Nombre de personnes par ligne
0,stade-tour-eiffel,12860,"[BIR HAKEIM, ALMA MARCEAU]","[6, 9]",2,6430.00
1,arena-champ-de-mars,8356,"[LA MOTTE-PICQUET-GRENELLE, LA MOTTE-PICQUET-G...","[6, 8, 10]",3,2785.33
2,grand-palais,8000,"[FRANKLIN D. ROOSEVELT, FRANKLIN D. ROOSEVELT]","[6, 9]",2,4000.00
3,invalides,8000,[INVALIDES],"[8, 13]",2,4000.00
4,pont-alexandre-iii,1000,[INVALIDES],"[8, 13]",2,500.00
5,trocadero,3349,"[IENA, TROCADERO]","[9, 6]",2,1674.50
6,concorde,35000,"[PALAIS-ROYAL, PALAIS-ROYAL, MADELEINE, MADELE...","[1, 7, 8, 12, 14, 3]",6,5833.33
7,stade-roland-garros,14962,"[PORTE DE SAINT-CLOUD, PORTE D'AUTEUIL]","[9, 10]",2,7481.00
8,arena-paris-sud,9000,"[PORTE DE VERSAILLES, BALARD, PORTE DE VANVES]","[12, 8, 13]",3,3000.00
9,arena-bercy,15000,"[BERCY, BERCY, GARE DE LYON]","[6, 14, 1]",3,5000.00


       LIBELLE_ARRET  00:00-01:00  01:00-02:00  02:00-03:00  03:00-04:00  \
0           abbesses        10.18         3.20         0.02         0.00   
1              ablon         0.26         0.03         0.00         0.02   
2      acheres grand         0.78          NaN          NaN          NaN   
3      acheres ville         0.76         0.31         0.13         0.09   
4      aeroport cdg1         0.76         0.14         0.01         0.04   
..               ...          ...          ...          ...          ...   
751           vosves          NaN          NaN          NaN          NaN   
752  vulaines sur se         0.11          NaN          NaN          NaN   
753           wagram         3.63         1.43         0.10         0.00   
754           yerres         0.73         0.11         0.01         0.05   
755   yvris noisy gd         0.20         0.04          NaN          NaN   

     04:00-05:00  05:00-06:00  06:00-07:00  07:00-08:00  08:00-09:00  ...  \
0         

/tmp/ipykernel_2430/3461944228.py:212: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


Copie de Nb de personnes + Arrêts + Lignes des lieux des événements

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# %% [markdown]
# Liste des lieux et url 

# %%

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_lines = {
            'Capacité': [],
            'Stations': []  # Création d'une liste pour stocker les stations
        }
        for paragraph in soup.find_all('p'):
            # Recherche des lignes contenant les Capacité spécifiques
            if ('Capacité' in paragraph.text or 
                'Capacité court Philippe-Chatrier' in paragraph.text or 
                'Volleyball' in paragraph.text or 
                'Pour la gymnastique rythmique' in paragraph.text):
                specific_lines['Capacité'].append(paragraph.text.strip())
                
            # Extraction des phrases après "desservie" ou "desservi"
            if 'desservie' in paragraph.text or 'desservi' in paragraph.text:
                sentences = paragraph.text.split('.')  # Séparation en phrases
                for sentence in sentences:
                    if 'desservie' in sentence or 'desservi' in sentence:
                        specific_lines['Stations'].append(sentence.strip())
        
        return specific_lines
    return None


# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Infos'])
# Expansion du dictionnaire pour créer les colonnes 'Capacité' et 'Stations'
df = pd.concat([df.drop(['Infos'], axis=1), df['Infos'].apply(pd.Series)], axis=1)


# %% [markdown]
# Compléter pour les données manquantes : Concorde 

# %%
# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[6, 1]=texte_places.strip()

# %% [markdown]
# Compléter pour les données manquantes : Arena Paris Sud

# %%
#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[8, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

# %%
def extraire_nombres(texte):
    nombres = re.findall(r'[-+]?\d*\.\d+|\d+', str(texte))  # Expression régulière pour les nombres entiers ou décimaux
    return ' '.join(nombres)  # Retourne les nombres extraits sous forme d'une chaîne séparée par des espaces

# Appliquer la fonction sur la colonne 'Capacité'
df['Nombre de personnes à l\'événément'] = df['Capacité'].apply(extraire_nombres)

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer "202" par une chaîne vide dans la colonne 'Nombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace('202', '')

# Supprimer les espaces dans la colonne 'NNombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace(' ', '')

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer les valeurs dans la colonne 'Nombres' aux lignes spécifiques
df.loc[2, 'Nombre de personnes à l\'événément'] = '8000'  # Remplacer la deuxième ligne par '8000'
df.loc[10, 'Nombre de personnes à l\'événément'] = '6700'  # Remplacer la 10ème ligne par '6700'
df.loc[14, 'Nombre de personnes à l\'événément'] = '6000'  # Remplacer la 14ème ligne par '6000'

# Fonction pour supprimer le zéro en tête des nombres
def remove_leading_zero(number):
    if number.startswith('0'):
        return number[1:]
    return number

# Appliquer la fonction à la colonne 'Nombres' pour supprimer les zéros en tête
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].astype(str).apply(remove_leading_zero)

# %%
arrêts=[['BIR HAKEIM', 'ALMA MARCEAU'],
        ['LA MOTTE-PICQUET-GRENELLE', 'LA MOTTE-PICQUET-GRENELLE', 'SEGUR'], 
        ['FRANKLIN D. ROOSEVELT', 'FRANKLIN D. ROOSEVELT'], 
        ['INVALIDES'], 
        ['INVALIDES'],
        ['IENA','TROCADERO'], 
        ['PALAIS-ROYAL','PALAIS-ROYAL','MADELEINE', 'MADELEINE', 'MADELEINE', 'OPERA'],
        ['PORTE DE SAINT-CLOUD','PORTE D\'AUTEUIL'], 
        ['PORTE DE VERSAILLES','BALARD','PORTE DE VANVES'], 
        ['BERCY','BERCY','GARE DE LYON'],
        ['PORTE DE LA CHAPELLE'],
        ['LA DEFENSE'],
        ['x'],
        ['SAINT-DENIS-PORTE DE PARIS'], 
        ['x'],
        ['x']]
 
df['Arrêts']=arrêts

# Fonction pour extraire les nombres après le mot "Métro"
def extract_numbers(text):
    matches = re.findall(r'Métro\s+(\d+)', str(text), flags=re.IGNORECASE)
    return matches

# Appliquer la fonction à la colonne 'Stations' pour créer une nouvelle colonne 'Nombres après Métro'
df['Lignes'] = df['Stations'].apply(extract_numbers)

# Compléter pour Franklin D
df.at[2, 'Lignes'] = [6, 9]

# %%
# Supprimer les colonnes 'Capacité' et 'Stations'
colonnes_a_supprimer = ['Capacité', 'Stations']
df = df.drop(colonnes_a_supprimer, axis=1)

# Calculer le nombre d'éléments dans chaque vecteur de la ligne 'Lignes'
df['Nombre de lignes'] = df['Lignes'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Convertir la colonne 'Nombre de personnes à l\'événément' en type numérique (int ou float)
df['Nombre de personnes à l\'événément'] = pd.to_numeric(df['Nombre de personnes à l\'événément'], errors='coerce')

# Créer la nouvelle colonne en calculant la division
df['Nombre de personnes par ligne'] = df['Nombre de personnes à l\'événément'] / df['Nombre de lignes']

# Définir l'option d'affichage pour éviter la notation scientifique
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# %%
# Identifier les indices des lignes contenant les valeurs spécifiques dans la colonne 'lieu'
indices_a_supprimer = df[df['Lieu'].isin(['centre-aquatique', 'site-escalade-bourget', 'arena-paris-nord'])].index

# Supprimer les lignes correspondantes
df = df.drop(indices_a_supprimer)

display(df)

df.to_excel('df.xlsx', index=False)

/tmp/ipykernel_2430/864975592.py:89: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texte_places = soup.find(text=lambda text: text and "places" in text)
/tmp/ipykernel_2430/864975592.py:110: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)


,Lieu,Nombre de personnes à l'événément,Arrêts,Lignes,Nombre de lignes,Nombre de personnes par ligne
0,stade-tour-eiffel,12860,"[BIR HAKEIM, ALMA MARCEAU]","[6, 9]",2,6430.00
1,arena-champ-de-mars,8356,"[LA MOTTE-PICQUET-GRENELLE, LA MOTTE-PICQUET-G...","[6, 8, 10]",3,2785.33
2,grand-palais,8000,"[FRANKLIN D. ROOSEVELT, FRANKLIN D. ROOSEVELT]","[6, 9]",2,4000.00
3,invalides,8000,[INVALIDES],"[8, 13]",2,4000.00
4,pont-alexandre-iii,1000,[INVALIDES],"[8, 13]",2,500.00
5,trocadero,3349,"[IENA, TROCADERO]","[9, 6]",2,1674.50
6,concorde,35000,"[PALAIS-ROYAL, PALAIS-ROYAL, MADELEINE, MADELE...","[1, 7, 8, 12, 14, 3]",6,5833.33
7,stade-roland-garros,14962,"[PORTE DE SAINT-CLOUD, PORTE D'AUTEUIL]","[9, 10]",2,7481.00
8,arena-paris-sud,9000,"[PORTE DE VERSAILLES, BALARD, PORTE DE VANVES]","[12, 8, 13]",3,3000.00
9,arena-bercy,15000,"[BERCY, BERCY, GARE DE LYON]","[6, 14, 1]",3,5000.00


Copie du programme fréquentation_metro

In [7]:
# %%
import pandas as pd
import numpy as np
import re
import difflib

# %% [markdown]
# Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

# %%
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)

# %% [markdown]
# Tableau avec station, ligne, correspondances...

# %%
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print(tableau_fréquentation)

# %%
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
display(tableau_fréquentation)

# %%
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# %%
# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

# %% [markdown]
# PROBLÈME À RÉGLER : CREER UNE COLONNE QUI COMPTE LE NOMBRE DE LIGNES

# %%
# Liste des colonnes à analyser
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Calcul du nombre de cases non vides pour chaque ligne
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].count(axis=1)

# %%
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']
print(tableau_fréquentation)

# %% [markdown]
# Création d'un tableau avec la moyenne sur la période des JO en 2022 du nombre de validation hebdomadire à chaque station

# %%
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(moyenne_par_arret)

# %% [markdown]
# Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne --> PROBLÈME À RÉGLER : CERTAINES CASES VIDES

# %%
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in x.values, axis=1)]
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

# %%
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

# %%
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table
        # Afficher le tableau mis à jour
        print(f"Tableau 'ligne numéro[{k_str}]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :")
        display(tableaux[f'ligne numéro[{k_str}]'])

/tmp/ipykernel_2430/2856490993.py:13: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


       LIBELLE_ARRET  00:00-01:00  01:00-02:00  02:00-03:00  03:00-04:00  \
0           abbesses        10.18         3.20         0.02         0.00   
1              ablon         0.26         0.03         0.00         0.02   
2      acheres grand         0.78          NaN          NaN          NaN   
3      acheres ville         0.76         0.31         0.13         0.09   
4      aeroport cdg1         0.76         0.14         0.01         0.04   
..               ...          ...          ...          ...          ...   
751           vosves          NaN          NaN          NaN          NaN   
752  vulaines sur se         0.11          NaN          NaN          NaN   
753           wagram         3.63         1.43         0.10         0.00   
754           yerres         0.73         0.11         0.01         0.05   
755   yvris noisy gd         0.20         0.04          NaN          NaN   

     04:00-05:00  05:00-06:00  06:00-07:00  07:00-08:00  08:00-09:00  ...  \
0         

,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,7,Métro,REPUBLIQUE,11079708,3,5,8,9.00,11.00,Paris,11.00
1,9,Métro,LA DEFENSE,9256802,1,NaN,NaN,NaN,NaN,Puteaux,NaN
2,13,Métro,HOTEL DE VILLE,7251729,1,11,NaN,NaN,NaN,Paris,4.00
3,30,Métro,VILLEJUIF-LOUIS ARAGON,5218070,7,NaN,NaN,NaN,NaN,Villejuif,NaN
4,40,Métro,PALAIS-ROYAL,4822599,1,7,NaN,NaN,NaN,Paris,1.00
...,...,...,...,...,...,...,...,...,...,...,...
357,286,Métro,PICPUS,931602,6,NaN,NaN,NaN,NaN,Paris,12.00
358,287,Métro,TEMPLE,891858,3,NaN,NaN,NaN,NaN,Paris,2.00
359,289,Métro,QUAI DE LA RAPEE,798728,5,NaN,NaN,NaN,NaN,Paris,12.00
360,299,Métro,DANUBE,400157,7bis,NaN,NaN,NaN,NaN,Paris,19.00


     Rang                 Station     Trafic Correspondance_1  \
0       7              REPUBLIQUE 2215941.60                3   
1       9              LA DEFENSE 1851360.40                1   
2      13          HOTEL DE VILLE 1450345.80                1   
3      30  VILLEJUIF-LOUIS ARAGON 1043614.00                7   
4      40            PALAIS-ROYAL  964519.80                1   
..    ...                     ...        ...              ...   
357   286                  PICPUS  186320.40                6   
358   287                  TEMPLE  178371.60                3   
359   289        QUAI DE LA RAPEE  159745.60                5   
360   299                  DANUBE   80031.40             7bis   
361   303       PRE-SAINT-GERVAIS   56525.20             7bis   

    Correspondance_2 Correspondance_3 Correspondance_4 Correspondance_5  \
0                  5                8              9.0             11.0   
1                nan              nan              nan              n

,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.32
1,ablon,1101.11
2,acheres grand,51.39
3,acheres ville,3012.05
4,aeroport cdg1,5050.95
...,...,...
733,vosves,14.89
734,vulaines sur se,42.89
735,wagram,3268.16
736,yerres,2743.00


...

,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,86,concorde,680243.80,5,23051055.60,0.03,9636.53,10.30,2.87,0.08,...,66.58,69.70,54.94,27.15,19.30,18.36,17.88,1927.31,963.65,105667.29
1,105,pasteur,605257.20,5,23051055.60,0.03,6307.74,7.42,2.12,0.13,...,43.50,44.12,35.49,20.42,12.83,11.10,10.56,1261.55,630.77,105667.29
2,233,abbesses,302776.80,5,23051055.60,0.01,2277.32,10.18,3.20,0.02,...,45.59,43.24,37.00,23.36,19.02,19.36,18.35,455.46,227.73,105667.29
3,280,trinite-d'estienne d'orves,210396.40,5,23051055.60,0.01,3454.79,12.18,6.56,0.22,...,51.19,54.76,49.49,33.64,26.85,21.44,16.85,690.96,345.48,105667.29
4,291,saint-georges,150311.00,5,23051055.60,0.01,1449.11,9.83,3.71,0.10,...,47.79,42.88,34.41,21.11,16.17,26.32,17.00,289.82,144.91,105667.29
5,296,assemblee nationale,122302.40,5,23051055.60,0.01,1602.26,14.18,8.04,0.10,...,65.30,64.65,41.95,25.79,16.80,15.93,18.23,320.45,160.23,105667.29
6,2,saint-lazare,6625676.80,5,23051055.60,0.29,127163.37,8.62,0.10,0.01,...,57.36,63.81,51.34,34.47,21.65,16.09,13.86,25432.67,12716.34,105667.29
7,2,saint-lazare,6625676.80,5,23051055.60,0.29,127163.37,8.06,1.80,0.14,...,85.45,92.97,77.22,51.26,27.60,21.59,14.81,25432.67,12716.34,105667.29
8,55,marcadet-poissonniers,796401.00,5,23051055.60,0.03,9417.42,4.49,1.81,0.10,...,43.07,42.94,37.58,27.00,16.68,10.82,7.56,1883.48,941.74,105667.29
9,67,convention,746950.00,5,23051055.60,0.03,7157.16,4.27,1.32,0.07,...,37.75,38.46,33.51,20.97,13.06,9.80,7.74,1431.43,715.72,105667.29


Tableau 'ligne numéro[13]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,54,basilique de saint-denis,798279.00,5,16957848.60,0.05,1889.26,1.98,0.30,0.05,...,39.92,33.92,25.82,14.12,8.43,6.61,4.61,377.85,188.93,37710.67
1,76,plaisance,704350.60,5,16957848.60,0.04,7378.74,4.13,1.31,0.03,...,39.72,38.27,32.66,22.15,12.53,8.94,7.01,1475.75,737.87,37710.67
2,155,malakoff-plateau de vanves,448464.00,5,16957848.60,0.03,4647.68,2.93,1.02,0.09,...,39.62,41.65,31.21,18.21,9.74,7.87,5.92,929.54,464.77,37710.67
3,158,guy moquet,441062.60,5,16957848.60,0.03,5378.58,5.08,2.09,0.06,...,37.81,36.78,33.77,20.83,12.24,9.86,8.41,1075.72,537.86,37710.67
4,187,champs-elysees-clemenceau,381801.00,5,16957848.60,0.02,4301.32,15.47,5.89,0.09,...,62.33,67.18,52.39,31.77,25.55,24.32,23.10,860.26,430.13,37710.67
5,189,les agnettes,378071.20,5,16957848.60,0.02,4123.32,2.65,0.77,0.04,...,34.57,31.37,25.58,17.10,10.05,7.16,5.00,824.66,412.33,37710.67
6,198,la fourche,365854.20,5,16957848.60,0.02,3468.05,7.98,3.66,0.13,...,40.81,40.02,37.61,24.24,15.98,12.67,11.38,693.61,346.81,37710.67
7,290,varenne,156539.40,5,16957848.60,0.01,1845.11,6.08,2.24,0.09,...,69.74,68.24,34.46,17.61,11.87,10.16,16.57,369.02,184.51,37710.67
8,35,chatillon-montrouge,1006802.40,5,16957848.60,0.06,11674.53,2.28,0.61,0.02,...,41.77,41.56,29.17,16.87,9.27,6.52,5.61,2334.91,1167.45,37710.67
9,39,mairie de saint-ouen,966162.00,5,16957848.60,0.06,12017.74,7.75,2.71,0.19,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.55,1201.77,37710.67


Tableau 'ligne numéro[14]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,39,mairie de saint-ouen,966162.00,5,15764240.20,0.06,12017.74,7.75,2.71,0.19,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.55,1201.77,47415.80
1,50,pont cardinet,833707.60,5,15764240.20,0.05,9483.26,3.67,1.64,0.11,...,43.22,45.32,37.99,23.12,14.08,7.68,6.32,1896.65,948.33,47415.80
2,56,pyramides,792539.60,5,15764240.20,0.05,8558.26,6.84,2.03,0.13,...,63.20,67.67,51.92,33.49,24.30,22.48,18.01,1711.65,855.83,47415.80
3,84,clichy saint-ouen,684170.40,5,15764240.20,0.04,9117.84,2.59,1.24,0.18,...,54.71,54.96,36.11,20.36,12.13,6.13,4.06,1823.57,911.78,47415.80
4,6,bibliotheque,2220894.80,5,15764240.20,0.14,6088.00,6.46,1.35,0.06,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.60,608.80,47415.80
5,31,olympiades,1042919.00,5,15764240.20,0.07,12997.14,2.74,1.23,0.06,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.43,1299.71,47415.80
6,3,gare de lyon,5728095.00,5,15764240.20,0.36,66037.95,2.16,0.01,0.00,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.59,6603.79,47415.80
7,3,gare de lyon,5728095.00,5,15764240.20,0.36,66037.95,18.38,4.31,0.30,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.59,6603.79,47415.80
8,61,bercy,776842.40,5,15764240.20,0.05,9023.95,NaN,NaN,NaN,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.79,902.39,47415.80
9,61,bercy,776842.40,5,15764240.20,0.05,9023.95,10.59,2.39,0.22,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.79,902.39,47415.80


Programme

In [8]:
def calculer_personnes_supplementaires(date_horaire_entree, numero_ligne, station):

    tableau_debut = pd.DataFrame()  # Initialisation du DataFrame vide
    tableau_fin = pd.DataFrame()    # Initialisation du DataFrame vide

    cle_tableau = f'ligne numéro[{numero_ligne}]'

    # Extraire l'horaire de l'entrée utilisateur
    heure_entree = date_horaire_entree[-5:]

    # Déterminer la plage horaire à laquelle appartient l'horaire d'entrée
    plage_horaire_entree = f"{heure_entree.split(':')[0]}:00:00 - {(int(heure_entree.split(':')[0])+1)%24:02d}:00:00"

    # Construire la chaîne de date avec la plage horaire
    date_plage_horaire = f"{date_horaire_entree[:10]} {plage_horaire_entree}"

    # Vérifier si la chaîne date_plage_horaire se trouve dans new_horaires_début_df
    if (date_plage_horaire in new_horaires_début_df.values) or (date_plage_horaire in new_horaires_fin_df.values): 
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_début_df
        lieux_correspondants_debut = new_horaires_début_df.loc[new_horaires_début_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_fin_df
        lieux_correspondants_fin = new_horaires_fin_df.loc[new_horaires_fin_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        
        lignes_correspondantes = []
        
        
        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_debut
        for lieu in lieux_correspondants_debut:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_debut=pd.concat([tableau_debut, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)
            

        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_fin
        for lieu in lieux_correspondants_fin:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_fin=pd.concat([tableau_fin, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)

        numero_trouve = False

        for sous_liste in lignes_correspondantes:
            if numero_ligne in sous_liste:
                numero_trouve = True
                break  # Arrête la boucle une fois que le numéro est trouvé

            # Vérifier si le numéro de ligne entré est présent dans les vecteurs des lignes correspondantes
        if numero_trouve:
            resultat_debut=0
            resultat_fin=0
            if not tableau_debut.empty :
                tableau_debut = tableau_debut[tableau_debut['Lignes'].apply(lambda x: numero_ligne in x)]
                resultat_debut = tableau_debut['Nombre de personnes par ligne'].sum()
            if not tableau_fin.empty :
                    tableau_fin = tableau_fin[tableau_fin['Lignes'].apply(lambda x: numero_ligne in x)]
                    resultat_fin = tableau_fin['Nombre de personnes par ligne'].sum()
            resultat=resultat_debut+resultat_fin
            nb_ligne=round(resultat)
            print(f"Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré  sur la ligne en raison des JO  est {nb_ligne}.")

            resultat_debut_2=0
            resultat_fin_2=0

            if not tableau_debut.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part du trafic de la station par rapport à toute la ligne'].values[0]
                tableau_debut['Pourcentage fréquentation'] = valeur
                resultat_debut_2=valeur*resultat_debut
            
            if not tableau_fin.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part du trafic de la station par rapport à toute la ligne'].values[0]
                tableau_fin['Pourcentage fréquentation'] = valeur
                resultat_fin_2=valeur*resultat_debut
            
            resultat2=resultat_debut_2+resultat_fin_2
            nb_station=round(resultat2)
            print(f"Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré à la station en raison des JO  est {nb_station}.")

        else:
            print(0)
    else:
        print(0)

In [9]:
calculer_personnes_supplementaires('2024-07-28 09:20', '1', 'bastille')

Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré  sur la ligne en raison des JO  est 17000.
Le nombre de personnes supplémentaires sur la plage horaire de l'horaire entré à la station en raison des JO  est 984.


In [10]:
# date_horaire_entree = input("Entrez la date et l'horaire (format YYYY-MM-JJ HH:MM) : ")
# numero_ligne = input("Entrez le numéro de la ligne : ")